In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D
model=Sequential()
model.add(Conv1D(filters=16,
                 kernel_size=3,
                 padding='same', # same can be replaced by valid as they are the same options
                 activation='relu',
                 strides=1,
                 input_shape=(100,300))) # you don't change the input shape the padding happens under the hood

In [1]:
# Building the CNN froms scratch while using custom raw data
import numpy as np
from keras.preprocessing import sequence # helps in padding input
from keras.models import Sequential # base keras model
from keras.layers import Dense,Dropout,Activation
from keras.layers import Conv1D, GlobalMaxPooling1D # Convolution layer and poolingg


In [2]:
# we will use the imdb dataset
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00


In [6]:
!pip install nlpia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.0/32.0 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.6/706.6 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.6 MB/s eta 0:00:00


In [3]:
from datasets import list_datasets
from datasets import load_dataset
imdb=load_dataset("imdb")
imdb

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [40]:
imdb["test"][2500]["label"]


0

In [14]:
# tokenization
import gensim.downloader as api
from nltk.tokenize import TreebankWordTokenizer
from gensim.models.keyedvectors import KeyedVectors
w2v_model=api.load("word2vec-google-news-300")
w2v_model.save_word2vec_format('word2vec-google-news-300.bin',binary=True)
# make sure you save the model by its name to enable it save properly



In [20]:
def tokenize_and_vectorize(dataset, word_vectors):
    """Tokenizes text data using TreebankWordTokenizer and vectorizes using provided word vectors.

    Args:
        dataset: Dataset containing text data, where each sample is a list/tuple of (text, label).
        word_vectors: Loaded Word2Vec model for vectorization.

    Returns:
        List of lists, where each inner list represents tokenized and vectorized text.
    """

    tokenizer = TreebankWordTokenizer()
    vectorized_data = []
    expected=[]

    for sample in dataset:
        text = sample[0]  # Access text from the second element of the sample
        tokens = tokenizer.tokenize(text)
        sample_vecs = []

        for token in tokens:
            try:
                sample_vecs.append(word_vectors[token])
            except KeyError:
                # Handle missing tokens:
                sample_vecs.append(np.zeros(word_vectors.vector_size))  # Pad with zeros

        vectorized_data.append(sample_vecs)

    return vectorized_data


In [23]:
word_vectors = KeyedVectors.load_word2vec_format("word2vec-google-news-300.bin", binary=True)


In [55]:
import datasets

# Get label names directly from the feature:
label_names = imdb["train"].features["label"].names

# Create the encoder:
encoder = datasets.ClassLabel(num_classes=len(label_names), names=label_names)

# Apply encoding to each split:
# Apply encoding to each split:
for split in imdb.keys():
    imdb[split] = imdb[split].map(
        lambda example: {"label": encoder.str2int(example["label"])},  # Use str2int method for encoding
        batched=True,
    )

# Access encoded labels:
encoded_label = imdb["train"][0]["label"]


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [25]:
vectorized_training = tokenize_and_vectorize(imdb["train"]["text"],word_vectors)

In [26]:
vectorized_testing=tokenize_and_vectorize(imdb["test"]["text"],word_vectors)

In [34]:
print(imdb["train"].features)

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}


In [57]:
# getting the labels
def collect_expected(dataset):
    """ Peel off the target values from the dataset """
    expected = []
    for sample in dataset:
        expected.append(sample['label'])  # Access the 'label' column directly
    return expected

expected_training = collect_expected(imdb["train"])

In [58]:
expected_testing=collect_expected(imdb["test"])